# Unicode Objects and Codecs

Marc-André Lemburg \<<mal@lemburg.com>\>

Georg Brandl \<<georg@python.org>\>

## Unicode Objects

Since the implementation of `393` in Python 3.3, Unicode objects
internally use a variety of representations, in order to allow handling
the complete range of Unicode characters while staying memory efficient.
There are special cases for strings where all code points are below 128,
256, or 65536; otherwise, code points must be below 1114112 (which is
the full Unicode range).

UTF-8 representation is created on demand and cached in the Unicode
object.

Note

The :c`Py_UNICODE` representation has been removed since Python 3.12
with deprecated APIs. See `623` for more information.

### Unicode Type

These are the basic Unicode object types used for the Unicode
implementation in Python:

> This is a typedef of :c`wchar_t`, which is a 16-bit type or 32-bit
> type depending on the platform.
>
> 3.3 In previous versions, this was a 16-bit type or a 32-bit type
> depending on whether you selected a "narrow" or "wide" Unicode version
> of Python at build time.

> This instance of :c`PyTypeObject` represents the Python Unicode type.
> It is exposed to Python code as `str`.

The following APIs are C macros and static inlined functions for fast
checks and access to internal read-only data of Unicode objects:

> Return true if the object *o* is a Unicode object or an instance of a
> Unicode subtype. This function always succeeds.

> Return true if the object *o* is a Unicode object, but not an instance
> of a subtype. This function always succeeds.

> Returns `0`. This API is kept only for backward compatibility.
>
> 3.3
>
> 3.10 This API does nothing since Python 3.12.

> Return the length of the Unicode string, in code points. *o* has to be
> a Unicode object in the "canonical" representation (not checked).
>
> 3.3

> Return one of the PyUnicode kind constants (see above) that indicate
> how many bytes per character this Unicode object uses to store its
> data. *o* has to be a Unicode object in the "canonical" representation
> (not checked).
>
> 3.3

> Return a void pointer to the raw Unicode buffer. *o* has to be a
> Unicode object in the "canonical" representation (not checked).
>
> 3.3

> Read a character from a Unicode object *o*, which must be in the
> "canonical" representation. This is less efficient than
> :c`PyUnicode_READ` if you do multiple consecutive reads.
>
> 3.3

> Return the maximum code point that is suitable for creating another
> string based on *o*, which must be in the "canonical" representation.
> This is always an approximation but more efficient than iterating over
> the string.
>
> 3.3

> Return `1` if the string is a valid identifier according to the
> language definition, section `identifiers`. Return `0` otherwise.
>
> 3.9 The function does not call :c`Py_FatalError` anymore if the string
> is not ready.

### Unicode Character Properties

Unicode provides many different character properties. The most often
needed ones are available through these macros which are mapped to C
functions depending on the Python configuration.

> Return `1` or `0` depending on whether *ch* is a whitespace character.

> Return `1` or `0` depending on whether *ch* is a lowercase character.

> Return `1` or `0` depending on whether *ch* is an uppercase character.

> Return `1` or `0` depending on whether *ch* is a titlecase character.

> Return `1` or `0` depending on whether *ch* is a linebreak character.

> Return `1` or `0` depending on whether *ch* is a decimal character.

> Return `1` or `0` depending on whether *ch* is a digit character.

> Return `1` or `0` depending on whether *ch* is a numeric character.

> Return `1` or `0` depending on whether *ch* is an alphabetic
> character.

> Return `1` or `0` depending on whether *ch* is an alphanumeric
> character.

> Return `1` or `0` depending on whether *ch* is a printable character.
> Nonprintable characters are those characters defined in the Unicode
> character database as "Other" or "Separator", excepting the ASCII
> space (0x20) which is considered printable. (Note that printable
> characters in this context are those which should not be escaped when
> `repr` is invoked on a string. It has no bearing on the handling of
> strings written to `sys.stdout` or `sys.stderr`.)

These APIs can be used for fast direct character conversions:

> Return the character *ch* converted to lower case.
>
> 3.3 This function uses simple case mappings.

> Return the character *ch* converted to upper case.
>
> 3.3 This function uses simple case mappings.

> Return the character *ch* converted to title case.
>
> 3.3 This function uses simple case mappings.

> Return the character *ch* converted to a decimal positive integer.
> Return `-1` if this is not possible. This function does not raise
> exceptions.

> Return the character *ch* converted to a single digit integer. Return
> `-1` if this is not possible. This function does not raise exceptions.

> Return the character *ch* converted to a double. Return `-1.0` if this
> is not possible. This function does not raise exceptions.

These APIs can be used to work with surrogates:

> Check if *ch* is a surrogate (`0xD800 <= ch <= 0xDFFF`).

> Check if *ch* is a high surrogate (`0xD800 <= ch <= 0xDBFF`).

> Check if *ch* is a low surrogate (`0xDC00 <= ch <= 0xDFFF`).

> Join two surrogate characters and return a single :c`Py_UCS4` value.
> *high* and *low* are respectively the leading and trailing surrogates
> in a surrogate pair. *high* must be in the range \[0xD800; 0xDBFF\]
> and *low* must be in the range \[0xDC00; 0xDFFF\].

### Creating and accessing Unicode strings

To create Unicode objects and access their basic sequence properties,
use these APIs:

> Create a new Unicode object. *maxchar* should be the true maximum code
> point to be placed in the string. As an approximation, it can be
> rounded up to the nearest value in the sequence 127, 255, 65535,
> 1114111.
>
> This is the recommended way to allocate a new Unicode object. Objects
> created using this function are not resizable.
>
> 3.3

> Create a Unicode object from the char buffer *u*. The bytes will be
> interpreted as being UTF-8 encoded. The buffer is copied into the new
> object. The return value might be a shared object, i.e. modification
> of the data is not allowed.
>
> This function raises `SystemError` when:
>
> -   *size* \< 0,
> -   *u* is `NULL` and *size* \> 0
>
> 3.12 *u* == `NULL` with *size* \> 0 is not allowed anymore.

> Create a Unicode object from a UTF-8 encoded null-terminated char
> buffer *u*.

> Take a C :c`printf`-style *format* string and a variable number of
> arguments, calculate the size of the resulting Python Unicode string
> and return a string with the values formatted into it. The variable
> arguments must be C types and must correspond exactly to the format
> characters in the *format* ASCII-encoded string. The following format
> characters are allowed:
>
> l
>
> | Format Characters | Type                     | Comment                                                                                                                                                                                   |
> |-------------------|---------------------|--------------------------------|
> | `%%`              | *n/a*                    | The literal % character.                                                                                                                                                                  |
> | `%c`              | int                      | A single character, represented as a C int.                                                                                                                                               |
> | `%d`              | int                      | Equivalent to `printf("%d")`.                                                                                                                                                             |
> | `%u`              | unsigned int             | Equivalent to `printf("%u")`.                                                                                                                                                             |
> | `%ld`             | long                     | Equivalent to `printf("%ld")`.                                                                                                                                                            |
> | `%li`             | long                     | Equivalent to `printf("%li")`.                                                                                                                                                            |
> | `%lu`             | unsigned long            | Equivalent to `printf("%lu")`.                                                                                                                                                            |
> | `%lld`            | long long                | Equivalent to `printf("%lld")`.                                                                                                                                                           |
> | `%lli`            | long long                | Equivalent to `printf("%lli")`.                                                                                                                                                           |
> | `%llu`            | unsigned long long       | Equivalent to `printf("%llu")`.                                                                                                                                                           |
> | `%zd`             | :c:type:\`Py_ssize_t\`   | Equivalent to `printf("%zd")`.                                                                                                                                                            |
> | `%zi`             | :c:type:\`Py_ssize_t\`   | Equivalent to `printf("%zi")`.                                                                                                                                                            |
> | `%zu`             | size_t                   | Equivalent to `printf("%zu")`.                                                                                                                                                            |
> | `%i`              | int                      | Equivalent to `printf("%i")`.                                                                                                                                                             |
> | `%x`              | int                      | Equivalent to `printf("%x")`.                                                                                                                                                             |
> | `%s`              | const char\*             | A null-terminated C character array.                                                                                                                                                      |
> | `%p`              | const void\*             | The hex representation of a C pointer. Mostly equivalent to `printf("%p")` except that it is guaranteed to start with the literal `0x` regardless of what the platform's `printf` yields. |
> | `%A`              | PyObject\*               | The result of calling `ascii`.                                                                                                                                                            |
> | `%U`              | PyObject\*               | A Unicode object.                                                                                                                                                                         |
> | `%V`              | PyObject\*, const char\* | A Unicode object (which may be `NULL`) and a null-terminated C character array as a second parameter (which will be used, if the first parameter is `NULL`).                              |
> | `%S`              | PyObject\*               | The result of calling :c`PyObject_Str`.                                                                                                                                                   |
> | `%R`              | PyObject\*               | The result of calling :c`PyObject_Repr`.                                                                                                                                                  |
>
> An unrecognized format character causes all the rest of the format
> string to be copied as-is to the result string, and any extra
> arguments discarded.
>
> Note
>
> The width formatter unit is number of characters rather than bytes.
> The precision formatter unit is number of bytes for `"%s"` and `"%V"`
> (if the `PyObject*` argument is `NULL`), and a number of characters
> for `"%A"`, `"%U"`, `"%S"`, `"%R"` and `"%V"` (if the `PyObject*`
> argument is not `NULL`).
>
> 3.2 Support for `"%lld"` and `"%llu"` added.
>
> 3.3 Support for `"%li"`, `"%lli"` and `"%zi"` added.
>
> 3.4 Support width and precision formatter for `"%s"`, `"%A"`, `"%U"`,
> `"%V"`, `"%S"`, `"%R"` added.

> Identical to :c`PyUnicode_FromFormat` except that it takes exactly two
> arguments.

> Return the length of the Unicode object, in code points.
>
> 3.3

> Read a character from a string. This function checks that *unicode* is
> a Unicode object and the index is not out of bounds, in contrast to
> :c`PyUnicode_READ_CHAR`, which performs no error checking.
>
> 3.3

> Copy the string *u* into a new UCS4 buffer that is allocated using
> :c`PyMem_Malloc`. If this fails, `NULL` is returned with a
> `MemoryError` set. The returned buffer always has an extra null code
> point appended.
>
> 3.3

> Copy an instance of a Unicode subtype to a new true Unicode object if
> necessary. If *obj* is already a true Unicode object (not a subtype),
> return the reference with incremented refcount.
>
> Objects other than Unicode or its subtypes will cause a `TypeError`.

### Locale Encoding

The current locale encoding can be used to decode text from the
operating system.

> Similar to :c`PyUnicode_DecodeLocaleAndSize`, but compute the string
> length using :c`strlen`.
>
> 3.3

> Encode a Unicode object to UTF-8 on Android and VxWorks, or to the
> current locale encoding on other platforms. The supported error
> handlers are `"strict"` and `"surrogateescape"` (`383`). The encoder
> uses `"strict"` error handler if *errors* is `NULL`. Return a `bytes`
> object. *unicode* cannot contain embedded null characters.
>
> Use :c`PyUnicode_EncodeFSDefault` to encode a string to the
> `filesystem encoding and error handler`.
>
> This function ignores the `Python UTF-8 Mode <utf8-mode>`.
>
> The :c`Py_EncodeLocale` function.
>
> 3.3
>
> 3.7 The function now also uses the current locale encoding for the
> `surrogateescape` error handler, except on Android. Previously,
> :c`Py_EncodeLocale` was used for the `surrogateescape`, and the
> current locale encoding was used for `strict`.

### File System Encoding

Functions encoding to and decoding from the `filesystem encoding and
error handler` (`383` and `529`).

To encode file names to `bytes` during argument parsing, the `"O&"`
converter should be used, passing :c`PyUnicode_FSConverter` as the
conversion function:

> ParseTuple converter: encode `str` objects -- obtained directly or
> through the `os.PathLike` interface -- to `bytes` using
> :c`PyUnicode_EncodeFSDefault`; `bytes` objects are output as-is.
> *result* must be a :c`PyBytesObject*` which must be released when it
> is no longer used.
>
> 3.1
>
> 3.6 Accepts a `path-like object`.

To decode file names to `str` during argument parsing, the `"O&"`
converter should be used, passing :c`PyUnicode_FSDecoder` as the
conversion function:

> ParseTuple converter: decode `bytes` objects -- obtained either
> directly or indirectly through the `os.PathLike` interface -- to `str`
> using :c`PyUnicode_DecodeFSDefaultAndSize`; `str` objects are output
> as-is. *result* must be a :c`PyUnicodeObject*` which must be released
> when it is no longer used.
>
> 3.2
>
> 3.6 Accepts a `path-like object`.

> Decode a string from the `filesystem encoding and error handler`.
>
> If you need to decode a string from the current locale encoding, use
> :c`PyUnicode_DecodeLocaleAndSize`.
>
> The :c`Py_DecodeLocale` function.
>
> 3.6 The `filesystem error handler <filesystem encoding and error
> handler>` is now used.

> Decode a null-terminated string from the `filesystem encoding and
> error handler`.
>
> If the string length is known, use
> :c`PyUnicode_DecodeFSDefaultAndSize`.
>
> 3.6 The `filesystem error handler <filesystem encoding and error
> handler>` is now used.

> Encode a Unicode object to the `filesystem encoding and error
> handler`, and return `bytes`. Note that the resulting `bytes` object
> can contain null bytes.
>
> If you need to encode a string to the current locale encoding, use
> :c`PyUnicode_EncodeLocale`.
>
> The :c`Py_EncodeLocale` function.
>
> 3.2
>
> 3.6 The `filesystem error handler <filesystem encoding and error
> handler>` is now used.

### wchar_t Support

:c`wchar_t` support for platforms which support it:

> Create a Unicode object from the :c`wchar_t` buffer *w* of the given
> *size*. Passing `-1` as the *size* indicates that the function must
> itself compute the length, using wcslen. Return `NULL` on failure.

> Copy the Unicode object contents into the :c`wchar_t` buffer *w*. At
> most *size* :c`wchar_t` characters are copied (excluding a possibly
> trailing null termination character). Return the number of :c`wchar_t`
> characters copied or `-1` in case of an error. Note that the resulting
> :c`wchar_t*` string may or may not be null-terminated. It is the
> responsibility of the caller to make sure that the :c`wchar_t*` string
> is null-terminated in case this is required by the application. Also,
> note that the :c`wchar_t*` string might contain null characters, which
> would cause the string to be truncated when used with most C
> functions.

> Convert the Unicode object to a wide character string. The output
> string always ends with a null character. If *size* is not `NULL`,
> write the number of wide characters (excluding the trailing null
> termination character) into *\*size*. Note that the resulting
> :c`wchar_t` string might contain null characters, which would cause
> the string to be truncated when used with most C functions. If *size*
> is `NULL` and the :c`wchar_t*` string contains null characters a
> `ValueError` is raised.
>
> Returns a buffer allocated by :c`PyMem_Alloc` (use :c`PyMem_Free` to
> free it) on success. On error, returns `NULL` and *\*size* is
> undefined. Raises a `MemoryError` if memory allocation is failed.
>
> 3.2
>
> 3.7 Raises a `ValueError` if *size* is `NULL` and the :c`wchar_t*`
> string contains null characters.

## Built-in Codecs

Python provides a set of built-in codecs which are written in C for
speed. All of these codecs are directly usable via the following
functions.

Many of the following APIs take two arguments encoding and errors, and
they have the same semantics as the ones of the built-in `str` string
object constructor.

Setting encoding to `NULL` causes the default encoding to be used which
is UTF-8. The file system calls should use :c`PyUnicode_FSConverter` for
encoding file names. This uses the
`filesystem encoding and error handler` internally.

Error handling is set by errors which may also be set to `NULL` meaning
to use the default handling defined for the codec. Default error
handling for all built-in codecs is "strict" (`ValueError` is raised).

The codecs all use a similar interface. Only deviations from the
following generic ones are documented for simplicity.

### Generic Codecs

These are the generic codec APIs:

### UTF-8 Codecs

These are the UTF-8 codec APIs:

> Create a Unicode object by decoding *size* bytes of the UTF-8 encoded
> string *s*. Return `NULL` if an exception was raised by the codec.

> Encode a Unicode object using UTF-8 and return the result as Python
> bytes object. Error handling is "strict". Return `NULL` if an
> exception was raised by the codec.

> Return a pointer to the UTF-8 encoding of the Unicode object, and
> store the size of the encoded representation (in bytes) in *size*. The
> *size* argument can be `NULL`; in this case no size will be stored.
> The returned buffer always has an extra null byte appended (not
> included in *size*), regardless of whether there are any other null
> code points.
>
> In the case of an error, `NULL` is returned with an exception set and
> no *size* is stored.
>
> This caches the UTF-8 representation of the string in the Unicode
> object, and subsequent calls will return a pointer to the same buffer.
> The caller is not responsible for deallocating the buffer. The buffer
> is deallocated and pointers to it become invalid when the Unicode
> object is garbage collected.
>
> 3.3
>
> 3.7 The return type is now `const char *` rather of `char *`.
>
> 3.10 This function is a part of the `limited API <stable>`.

> As :c`PyUnicode_AsUTF8AndSize`, but does not store the size.
>
> 3.3
>
> 3.7 The return type is now `const char *` rather of `char *`.

### UTF-32 Codecs

These are the UTF-32 codec APIs:

> Return a Python byte string using the UTF-32 encoding in native byte
> order. The string always starts with a BOM mark. Error handling is
> "strict". Return `NULL` if an exception was raised by the codec.

### UTF-16 Codecs

These are the UTF-16 codec APIs:

> Return a Python byte string using the UTF-16 encoding in native byte
> order. The string always starts with a BOM mark. Error handling is
> "strict". Return `NULL` if an exception was raised by the codec.

### UTF-7 Codecs

These are the UTF-7 codec APIs:

> Create a Unicode object by decoding *size* bytes of the UTF-7 encoded
> string *s*. Return `NULL` if an exception was raised by the codec.

### Unicode-Escape Codecs

These are the "Unicode Escape" codec APIs:

> Encode a Unicode object using Unicode-Escape and return the result as
> a bytes object. Error handling is "strict". Return `NULL` if an
> exception was raised by the codec.

### Raw-Unicode-Escape Codecs

These are the "Raw Unicode Escape" codec APIs:

> Encode a Unicode object using Raw-Unicode-Escape and return the result
> as a bytes object. Error handling is "strict". Return `NULL` if an
> exception was raised by the codec.

### Latin-1 Codecs

These are the Latin-1 codec APIs: Latin-1 corresponds to the first 256
Unicode ordinals and only these are accepted by the codecs during
encoding.

> Create a Unicode object by decoding *size* bytes of the Latin-1
> encoded string *s*. Return `NULL` if an exception was raised by the
> codec.

> Encode a Unicode object using Latin-1 and return the result as Python
> bytes object. Error handling is "strict". Return `NULL` if an
> exception was raised by the codec.

### ASCII Codecs

These are the ASCII codec APIs. Only 7-bit ASCII data is accepted. All
other codes generate errors.

> Create a Unicode object by decoding *size* bytes of the ASCII encoded
> string *s*. Return `NULL` if an exception was raised by the codec.

> Encode a Unicode object using ASCII and return the result as Python
> bytes object. Error handling is "strict". Return `NULL` if an
> exception was raised by the codec.

### Character Map Codecs

This codec is special in that it can be used to implement many different
codecs (and this is in fact what was done to obtain most of the standard
codecs included in the `encodings` package). The codec uses mappings to
encode and decode characters. The mapping objects provided must support
the `__getitem__` mapping interface; dictionaries and sequences work
well.

These are the mapping codec APIs:

> Encode a Unicode object using the given *mapping* object and return
> the result as a bytes object. Error handling is "strict". Return
> `NULL` if an exception was raised by the codec.
>
> The *mapping* object must map Unicode ordinal integers to bytes
> objects, integers in the range from 0 to 255 or `None`. Unmapped
> character ordinals (ones which cause a `LookupError`) as well as
> mapped to `None` are treated as "undefined mapping" and cause an
> error.

The following codec API is special in that maps Unicode to Unicode.

> Translate a string by applying a character mapping table to it and
> return the resulting Unicode object. Return `NULL` if an exception was
> raised by the codec.
>
> The mapping table must map Unicode ordinal integers to Unicode ordinal
> integers or `None` (causing deletion of the character).
>
> Mapping tables need only provide the `__getitem__` interface;
> dictionaries and sequences work well. Unmapped character ordinals
> (ones which cause a `LookupError`) are left untouched and are copied
> as-is.
>
> *errors* has the usual meaning for codecs. It may be `NULL` which
> indicates to use the default error handling.

### MBCS codecs for Windows

These are the MBCS codec APIs. They are currently only available on
Windows and use the Win32 MBCS converters to implement the conversions.
Note that MBCS (or DBCS) is a class of encodings, not just one. The
target encoding is defined by the user settings on the machine running
the codec.

> Create a Unicode object by decoding *size* bytes of the MBCS encoded
> string *s*. Return `NULL` if an exception was raised by the codec.

> Encode a Unicode object using MBCS and return the result as Python
> bytes object. Error handling is "strict". Return `NULL` if an
> exception was raised by the codec.

> Encode the Unicode object using the specified code page and return a
> Python bytes object. Return `NULL` if an exception was raised by the
> codec. Use :c`CP_ACP` code page to get the MBCS encoder.
>
> 3.3

### Methods & Slots

## Methods and Slot Functions

The following APIs are capable of handling Unicode objects and strings
on input (we refer to them as strings in the descriptions) and return
Unicode objects or integers as appropriate.

They all return `NULL` or `-1` if an exception occurs.

> Concat two strings giving a new Unicode string.

> Split a string giving a list of Unicode strings. If *sep* is `NULL`,
> splitting will be done at all whitespace substrings. Otherwise, splits
> occur at the given separator. At most *maxsplit* splits will be done.
> If negative, no limit is set. Separators are not included in the
> resulting list.

> Split a Unicode string at line breaks, returning a list of Unicode
> strings. CRLF is considered to be one line break. If *keepend* is `0`,
> the line break characters are not included in the resulting strings.

> Join a sequence of strings using the given *separator* and return the
> resulting Unicode string.

> Compare two strings and return `-1`, `0`, `1` for less than, equal,
> and greater than, respectively.
>
> This function returns `-1` upon failure, so one should call
> :c`PyErr_Occurred` to check for errors.

> Compare a Unicode object, *uni*, with *string* and return `-1`, `0`,
> `1` for less than, equal, and greater than, respectively. It is best
> to pass only ASCII-encoded strings, but the function interprets the
> input string as ISO-8859-1 if it contains non-ASCII characters.
>
> This function does not raise exceptions.

> Rich compare two Unicode strings and return one of the following:
>
> -   `NULL` in case an exception was raised
> -   `Py_True` or `Py_False` for successful comparisons
> -   `Py_NotImplemented` in case the type combination is unknown
>
> Possible values for *op* are `Py_GT`, `Py_GE`, `Py_EQ`, `Py_NE`,
> `Py_LT`, and `Py_LE`.

> Return a new string object from *format* and *args*; this is analogous
> to `format % args`.

> Check whether *element* is contained in *container* and return true or
> false accordingly.
>
> *element* has to coerce to a one element Unicode string. `-1` is
> returned if there was an error.

> Intern the argument *\*string* in place. The argument must be the
> address of a pointer variable pointing to a Python Unicode string
> object. If there is an existing interned string that is the same as
> *\*string*, it sets *\*string* to it (decrementing the reference count
> of the old string object and incrementing the reference count of the
> interned string object), otherwise it leaves *\*string* alone and
> interns it (incrementing its reference count). (Clarification: even
> though there is a lot of talk about reference counts, think of this
> function as reference-count-neutral; you own the object after the call
> if and only if you owned it before the call.)

> A combination of :c`PyUnicode_FromString` and
> :c`PyUnicode_InternInPlace`, returning either a new Unicode string
> object that has been interned, or a new ("owned") reference to an
> earlier interned string object with the same value.